In [233]:
# import package 
import pandas as pd
import numpy as np
from numpy import linalg as LA
import copy
import plotly.figure_factory as ff
import plotly.plotly as py

In [211]:
# creating data with size 1000

player_id=["Player_"+str(x+1) for x in np.arange(1000)]

# create a sample csv file called player for use 
players = pd.DataFrame({'Player_id':player_id ,
                  'Skill_A': np.random.randint(low=1, high=10, size=(1000,)),
                   'Skill_B':np.random.randint(low=1, high=10, size=(1000,)),
                   'Skill_C':np.random.randint(low=1, high=10, size=(1000,))
                  })

In [ ]:
#export data to csv
players=players.set_index('Player_id')
players.to_csv("players.csv")

In [212]:
#import data to dataframe called players
players=pd.read_csv("players.csv")

In [8]:
############algorithm for getting Kmeans function. Kmeans function 
############can get the best the criteria on which we can use to split the players into teams
def get_labels(points, centroids):
    labels = []
    for point in points:
        distances = [LA.norm(point - centroid) for centroid in centroids]
        labels.append(np.argmin([distances]))
    return np.array(labels)

def get_centroids(points, labels, k):
    centroids = []
    for label in range(k):
        centroids.append(np.mean(points[np.where(labels==label)], axis=0))
    return centroids
        
def converged(old_centroids, centroids):
    return np.array_equal(old_centroids, centroids)




def kmeans(points, k):
    # Initialize centroids randomly
    centroids = points[np.random.randint(low=0, high=points.shape[0], size=k)]
    
    # keeping track of the number of iterations
    iterations = 0
    old_centroids = None
    
    while not converged(old_centroids, centroids):
        old_centroids = centroids
        iterations += 1
        
        # Assign labels to each datapoint based on centroids
        labels = get_labels(points, centroids)
        
        # Assign centroids based on datapoint labels
        centroids = get_centroids(points, labels, k)
        
    # We can get the labels too by calling getLabels(dataSet, centroids)
    return centroids

In [9]:
#############################################################

def get_assign_team(final_label,k):
      return [i for i in np.arange(k) if list(final_label).count(i)>len(points)//k][0]


def get_index_final_label(co_final_label,assign_team):
    #co_final_label=copy.copy(final_label)
    index=[i for i, e in enumerate(co_final_label) if e == assign_team]
    return index

def get_index_to_replace(co_final_label,index,assign_team):
    co_final_label=co_final_label*-1
    for x in index:
        co_final_label[x]=LA.norm(points[x] - centers[assign_team])
    relabel=[]
    for i in np.arange(len(index)-len(points)//team):
        max_index=list(co_final_label).index(max(co_final_label))
        relabel.append(max_index)
        co_final_label[max_index]=-100
    return relabel

def get_players_to_replace(points,relabel):
    repoints=[]
    for x in relabel:
        repoints.append(points[x])
    return repoints

def get_new_team_label(repoints,centers):
    replace_label=get_labels(repoints, centers)
    return replace_label


def update_final_label(final_label,index_to_replace,new_team_label):
    for x in np.arange(len(index_to_replace)):
         final_label[index_to_replace[x]]=new_team_label[x]
    return final_label





##finally, this is the function which reteams the players for equality in team
def update(final_label,team):
    for i in np.arange(team-1):
        
        assign_team=get_assign_team(final_label,team)
    #
        co_final_label=copy.copy(final_label)
    #
        index_final_label=get_index_final_label(co_final_label,assign_team)
    #
        index_to_replace=get_index_to_replace(co_final_label,index_final_label,assign_team)
    #
        players_to_replace=get_players_to_replace(points,index_to_replace)
    #
        centers[assign_team]=np.array([-20,-20,-20])
    #
        new_team_label=get_new_team_label(players_to_replace,centers)
    #
        update=update_final_label(final_label,index_to_replace,new_team_label)
    return update

In [237]:
########## this part is the functions that for the case have extra data( 1001 players, but 5 teams!)
def show_table(mydata):
    table = ff.create_table(mydata)
    return py.iplot(table, filename='')

def fil_data(points,team):
    res=len(points)%team
    div=len(points)//team
    extra_player=[]
    if res>0:
        l=list(points)
        for i in np.arange(res):
            l.pop(div+i)
            extra_player.append(l[div+i])
        points=np.array(l)
    return [points, extra_player]

def add_extra_player_label(extra_player,team_label_tabel):
    if len(extra_player)>0:
        a=list(team_label_tabel)
        extra_label=get_new_team_label(extra_player,centers)
        for i in extra_label:
            a.append(i)
        team_label_tabel=np.array(a)
    return team_label_tabel

In [ ]:
########## this part is the function for generating tables as required 
def show_table(mydata):
    table = ff.create_table(mydata)
    return py.iplot(table, filename='')

def show_table(mydata):
    table = ff.create_table(mydata)
    return py.iplot(table, filename='')

def add_label(players,team_label_tabel):
    players['Label'] = pd.Series(team_label_tabel, index=players.index)
    #return mydata.sort_values(by=['cluster'])
    return players

def split(label_players):
    n=[]
    for i in np.arange(int(team)):
        n.append(label_players[label_players.Label==i])
    return n

def agg(semi_team):
    team_n=semi_team.Label.min()
    df2 = pd.DataFrame({'Player_id':['Total'],
                'Skill_A':semi_team.Skill_A.sum(),
                'Skill_B':semi_team.Skill_B.sum(),
                'Skill_C':semi_team.Skill_C.sum(),
                     })
    n=semi_team.drop(columns=['Label']).append(df2)
    #n.set_index('Player_id')
    n.rename(columns={'Player_id':'Team_'+str(team_n+1)}, inplace=True)
    #n.set_index('Player_id')
    #n.index.names = ['Team_'+str(team_n+1)]
    return n

def player_in_team(label_players):
    label_players['Team']=label_players.Label+1
    mydata_team=label_players[['Team',"Label"]].groupby(['Team']).count()
    mydata_team.rename(columns={'Label':'num_of_player'}, inplace=True)
    return mydata_team

def add_to_players(a,b,c,players):
    centers=kmeans(main_player, team)
    new=get_new_team_label([[int(a),int(b),int(c)]],centers)
    players.set_index('Player_id')
    df2 = pd.DataFrame({'Player_id':['player_'+str(players.Player_id.count()+1)],
                'Skill_A':int(a),
                'Skill_B':int(b),
                'Skill_C':int(c),
                 'Label':new
                   })
    n=players.append(df2)[["Player_id",'Skill_A','Skill_B','Skill_C','Label']]
    return n  
    

In [250]:
#have a look at data
#show_table(players.head())
players.head()

,Player_id,Skill_A,Skill_B,Skill_C,Label
0,Player_1,2,7,6,3
1,Player_2,3,2,4,3
2,Player_3,8,7,8,1
3,Player_4,8,1,7,3
4,Player_5,1,9,7,3


In [214]:
#console to interact
team = input("How many teams do you want?")
if int(team)<0:
    print('Please renter a numebr')
team=int(team)

How many teams do you want?5


In [215]:
#since number of players are equal if we have 1001 players and wanna 5 teams, 
# 1000 are main_player and the 1 are extra_playe
player = players.values[:,1:]
main_player=fil_data(player,team)[0]
extra_player=fil_data(player,team)[1]

In [216]:
#use kmeans() function to get standard of clustering each team
# use get_final_labels() to get the labels of team players are belonging for main players(numbers of each team are different)
centers=kmeans(main_player, team)
final_label=get_labels(main_player,centers)

In [217]:
# sue update() function to reteam the players based on the result of get_final_labels() functions
# in order to make number of players are equal
team_label_tabel=update(final_label,team)

In [218]:
# check equality of number of players
for i in np.arange(team):
    print (list(team_label_tabel).count(i))

200
200
200
200
200


In [219]:
# in case we have extra players( for example 1001 players but 5 teams)
#add the extra players to the team based on given algorithm 
team_label_tabel=add_extra_player_label(extra_player,team_label_tabel)

In [220]:
# check again, since we only have 1000 players, there is no difference 
for i in np.arange(team):
    print (list(team_label_tabel).count(i))

200
200
200
200
200


In [ ]:
# show the splited tables

In [222]:
label_players=add_label(players,team_label_tabel)

In [223]:
list_of_team=split(label_players)
agg(list_of_team[1]).tail()

In [251]:
#show_table(agg(list_of_team[1]).tail())
agg(list_of_team[1]).tail()

,Team_2,Skill_A,Skill_B,Skill_C
980,Player_981,7,7,1
985,Player_986,9,7,9
996,Player_997,8,8,8
998,Player_999,7,6,4
0,Total,1498,1366,1035


In [252]:
#show_table(agg(list_of_team[2]).tail())
agg(list_of_team[2]).tail()

,Team_3,Skill_A,Skill_B,Skill_C
981,Player_982,1,1,7
983,Player_984,2,1,4
988,Player_989,4,2,6
999,Player_1000,4,4,5
0,Total,903,628,1401


In [253]:
#show_table(agg(list_of_team[4]).tail())
agg(list_of_team[4]).tail()

,Team_5,Skill_A,Skill_B,Skill_C
961,Player_962,8,3,2
974,Player_975,5,4,3
989,Player_990,6,6,5
994,Player_995,6,4,3
0,Total,1325,588,686


In [156]:
#console interacte
x=input('Do you want to know how many players are in each team? YES or NO')
if x=="YES":
    print(player_in_team(label_players))
else:
    print("okay, maybe later")

Do you want to know how many players are in each team? YES or NOYES
      num_of_player
Team               
1               200
2               200
3               200
4               200
5               200


In [246]:
# it will show the players that have been just add are belongs to which team!
x=input('Do you want to add more players? YES or NO')
if x=="NO":
    print('Okay, Maybe later')
if x=='YES':
    print('your player_id is'+ '  player_'+str(players.Player_id.count()+1))
    a=input('Please enter Skill_A score')
    b=input('Please enter Skill_B score')
    c=input('Please enter Skill_C score')
    print('this player has been add to the following team')
    players=add_to_players(a,b,c,players)
    ho1=split(players)
    agg(ho1[1]).tail()
    print(agg(ho1[1]).tail())
    

Do you want to add more players? YES or NOYES
your player_id is  player_1001
Please enter Skill_A score7
Please enter Skill_B score8
Please enter Skill_C score9
this player has been add to the following team
          Team_2  Skill_A  Skill_B  Skill_C
985   Player_986        9        7        9
996   Player_997        8        8        8
998   Player_999        7        6        4
0    player_1001        7        8        9
0          Total     1505     1374     1044


In [254]:
#the 1001 player has been added to data 
#show_table(players.tail())
players.tail()

,Player_id,Skill_A,Skill_B,Skill_C,Label
996,Player_997,8,8,8,1
997,Player_998,1,3,3,0
998,Player_999,7,6,4,1
999,Player_1000,4,4,5,2
0,player_1001,7,8,9,1
